In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft,fft2, fftshift
from skimage.io import imread
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset

In [ ]:
# sino=np.load('/mnt/DATA/EE21B040/Downloads/sinograms_3W_test/ID_0a1f2c77b.npy')

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import os

class CustomDataset(Dataset):
    def __init__(self, img_folder, csv_file,transform=None):
        """
        Args:
            img_folder (str): Path to the folder containing images.
            csv_file (str): Path to the CSV file with image names and labels.
            transform (callable, optional): Optional transform to be applied on the image.
        """
        self.img_folder = img_folder
        self.df = pd.read_csv(csv_file)
        self.transform = transform

        # Extract image file names and labels from the CSV file
        self.img_files = self.df['Image'].values
        self.labels = self.df['any'].values

    def __len__(self):
        # Return the length of the dataset (number of samples)
        return len(self.img_files)
    def __getitem__(self, index):
    # Load the image file
        img_filename = self.img_files[index]
        img_path = os.path.join(self.img_folder, f"{img_filename}.npy")  # Construct the .npy file path

    # Load the image as a NumPy array from the .npy file
        image = np.load(img_path)
        num_angles = image.shape[0]
        num_channels = image.shape[2]
        for angle_index in range(num_angles):
                # Iterate over each channel
            for channel_index in range(num_channels):
                    # Apply Fourier transform to the current channel at the current angle
                image[angle_index, :, channel_index] = abs(fft(image[angle_index, :, channel_index]))

    # Convert the NumPy array to a PyTorch tensor

        image = torch.from_numpy(image).float()
        image=image.permute(2,0,1)

    # Apply transformations if provided
        if self.transform:
            image = self.transform(image)

    # Get the label
        label = self.labels[index]

    # Convert the label to a PyTorch tensor (if necessary)
        #label = torch.tensor(label, dtype=torch.float32)
        label = torch.tensor([label], dtype=torch.float32)

        return image, label

        if self.transform:
            image = self.transform(image)


In [ ]:
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler


train_img_folder ='/mnt/DATA/sinograms_3W_train'  # Replace with your image folder path
train_csv_file = '/mnt/DATA/ee21b040/Desktop/Chandra/DLH/train_x.csv'  # Replace with your CSV file path
train_dataset = CustomDataset(train_img_folder, train_csv_file)

test_img_folder ='/mnt/DATA/EE21B040/Downloads/sinograms_3W_test' # Replace with your image folder path
test_csv_file = '/mnt/DATA/ee21b040/Desktop/Chandra/DLH/test_x.csv' # Replace with your CSV file path
test_dataset = CustomDataset(test_img_folder, test_csv_file)
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=True,drop_last=True)

# Set the random seed for reproducibility (optional)
torch.manual_seed(42)

# Get the length of your train_dataset
dataset_length = len(train_dataset)

# Generate a list of indices from 0 to dataset_length
indices = list(range(dataset_length))

# Shuffle the indices
torch.random.manual_seed(42)  # Set random seed for reproducibility
# torch.random.shuffle(indices)
indices = torch.randperm(dataset_length)
indices = indices.tolist()
# Select 20,000 random indices
selected_indices = indices[:75000]

# Create a SubsetRandomSampler using the selected indices
subset_sampler = SubsetRandomSampler(selected_indices)

train_loader = DataLoader(train_dataset, batch_size=50, sampler=subset_sampler,drop_last=True)
valid_img_folder ='/mnt/DATA/sinograms_3W_train'  # Replace with your image folder path
valid_csv_file = '/mnt/DATA/ee21b040/Desktop/Chandra/DLH/valid_x.csv' # Replace with your CSV file path
valid_dataset = CustomDataset(valid_img_folder, valid_csv_file)
valid_loader = DataLoader(valid_dataset, batch_size=50, shuffle=True,drop_last=True)

In [ ]:
len(valid_dataset)

25783

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BinaryClassificationCNN(nn.Module):
    def __init__(self):
        super(BinaryClassificationCNN, self).__init__()

        # Define convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 256, kernel_size=3, stride=1, padding=1)

        # Define pooling layers
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Run a sample image through the conv and pool layers to find the correct input size for fc1
        # Assuming input shape is (1, 3, 360, 362)
        sample_input = torch.randn(1, 3, 360, 362)
        sample_output = self.pool(F.relu(self.conv1(sample_input)))
        sample_output = self.pool(F.relu(self.conv2(sample_output)))
        sample_output = self.pool(F.relu(self.conv3(sample_output)))
        fc1_input_size = sample_output.numel()  # Calculate input size

        # Define fully connected layers with the correct input size
        self.fc1 = nn.Linear(fc1_input_size, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 1)

    def forward(self, x):
        # Convolutional layers with ReLU activation and pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        # Flatten the tensor for fully connected layers
        # x = x.view(x.size(0), -1)
        x = x.reshape(x.size(0), -1)


        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        # Output layer for binary classification using sigmoid activation
        x = self.fc3(x)

        return x


In [ ]:
for images, labels in train_loader:
        print(images.shape)
        break

torch.Size([50, 3, 360, 362])


In [ ]:
len(valid_loader)

515

In [ ]:
# Check if CUDA (GPU) is available
model = BinaryClassificationCNN()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Instantiate the model and move it to the device
model = model.to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # For binary classification
optimizer = optim.Adam(model.parameters(), lr=0.0001)



# Define the ModelCheckpoint function to save the model every 10 epochs
def save_checkpoint(epoch):
    torch.save(model.state_dict(), f'model_epoch_{epoch}.pth')

import time
import time

# Training loop
num_epochs = 15
for epoch in range(num_epochs):
    model.train()

    # Start the timer for the epoch
    start_time = time.time()

    train_loss = 0.0
    train_correct = 0
    train_total = 0

    # Training loop
    for i, (images, labels) in enumerate(train_loader):
        # Move images and labels to the device (GPU or CPU)
        # images = images.permute(0, 3, 1, 2)
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)

        # Reshape labels to match model output shape
        labels = labels.view(-1, 1)

        # Calculate loss
        loss = criterion(outputs, labels.float())
        train_loss += loss.item()

        # Calculate the number of correct predictions
        predictions = (outputs >= 0.5).float()  # Apply threshold to outputs
        train_correct += (predictions == labels.float()).sum().item()
        train_total += labels.size(0)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print batch progress
        if (i + 1) % 100 == 0:  # Modify this line to adjust the frequency of progress printing
            print(f'Epoch {epoch + 1}, Batch {i + 1}/{len(train_loader)}, Loss: {loss.item()}')

    # Calculate training accuracy
    train_accuracy = train_correct / train_total

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            # Move images and labels to the device (GPU or CPU)

            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            labels = labels.view(-1, 1)

            # Calculate loss
            loss = criterion(outputs, labels.float())
            val_loss += loss.item()

            # Calculate the number of correct predictions
            predictions = (outputs >= 0.5).float()  # Apply threshold to outputs
            val_correct += (predictions == labels.float()).sum().item()
            val_total += labels.size(0)

    # Calculate average validation loss
    val_loss /= len(valid_loader)

    # Calculate validation accuracy
    val_accuracy = val_correct / val_total

    # Print progress
    print(f'Epoch {epoch + 1}/{num_epochs}, '
          f'Train Loss: {train_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, '
          f'Validation Loss: {val_loss:.4f}, '
          f'Validation Accuracy: {val_accuracy:.4f}')

    # Calculate and print the time taken for the epoch
    end_time = time.time()
    epoch_time = end_time - start_time
    print(f'Epoch {epoch + 1} completed in {epoch_time:.2f} seconds')

    # Save the model every 10 epochs
    if (epoch + 1) % 10 == 0:
        save_checkpoint(epoch + 1)

print("Training complete!")

Using device: cuda
Epoch 1, Batch 100/1500, Loss: 0.5103228688240051
Epoch 1, Batch 200/1500, Loss: 0.5002064108848572
Epoch 1, Batch 300/1500, Loss: 0.4453103244304657
Epoch 1, Batch 400/1500, Loss: 0.5725246071815491
Epoch 1, Batch 500/1500, Loss: 0.5299974083900452
Epoch 1, Batch 600/1500, Loss: 0.46316078305244446
Epoch 1, Batch 700/1500, Loss: 0.47291257977485657
Epoch 1, Batch 800/1500, Loss: 0.5011010766029358
Epoch 1, Batch 900/1500, Loss: 0.47148871421813965
Epoch 1, Batch 1000/1500, Loss: 0.4746263027191162
Epoch 1, Batch 1100/1500, Loss: 0.5872865915298462
Epoch 1, Batch 1200/1500, Loss: 0.4974919855594635
Epoch 1, Batch 1300/1500, Loss: 0.5020282864570618
Epoch 1, Batch 1400/1500, Loss: 0.4351782202720642
Epoch 1, Batch 1500/1500, Loss: 0.45801085233688354
Epoch 1/15, Train Loss: 0.5712, Train Accuracy: 0.7156, Validation Loss: 0.4792, Validation Accuracy: 0.7021
Epoch 1 completed in 4802.10 seconds
Epoch 2, Batch 100/1500, Loss: 0.4278680384159088
Epoch 2, Batch 200/1500, 

In [ ]:
model.eval()  # Set the model to evaluation mode

test_loss = 0.0
test_correct = 0
test_total = 0

with torch.no_grad():
    for images, labels in test_loader:
        # Move images and labels to the device (GPU or CPU)
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)

        # Reshape labels to match model output shape
        labels = labels.view(-1, 1)

        # Calculate loss
        loss = criterion(outputs, labels.float())
        test_loss += loss.item()

        # Calculate the number of correct predictions
        predictions = (outputs >= 0.5).float()  # Apply threshold to outputs
        test_correct += (predictions == labels.float()).sum().item()
        test_total += labels.size(0)

# Calculate average test loss
test_loss /= len(test_loader)

# Calculate test accuracy
test_accuracy = test_correct / test_total

# Print test loss and accuracy
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


Test Loss: 1.8267, Test Accuracy: 0.7154


: 